In [1]:
import gdsfactory
import matplotlib
import pandas
import glayout

In [12]:
from glayout.flow.pdk.gf180_mapped import gf180_mapped_pdk
from glayout.flow.pdk.mappedpdk import MappedPDK 
from gdsfactory import Component
from gdsfactory.components import rectangle

def via(PDK: MappedPDK):
    via_dimension = PDK.get_grule('via1')['width']
    metal1_dimension = via_dimension + 2*PDK.get_grule('via1', 'met1')['min_enclosure']
    metal2_dimension = via_dimension + 2*PDK.get_grule('via1', 'met2')['min_enclosure']

    via_layer = PDK.get_glayer('via1')
    metal1_layer = PDK.get_glayer('met1')
    metal2_layer = PDK.get_glayer('met2')

    top_level = Component(name="via")
    top_level << rectangle(size=(via_dimension, via_dimension), layer=via_layer, centered=True)
    top_level << rectangle(size=(metal1_dimension, metal1_dimension), layer=metal1_layer, centered=True)
    top_level << rectangle(size=(metal2_dimension, metal2_dimension), layer=metal2_layer, centered=True)

    return top_level
via_component = via(PDK=gf180_mapped_pdk)
via_component.show()

2024-06-24 22:39:12.521 | INFO     | gdsfactory.klive:show:55 - Message from klive: {"version": "0.3.2", "type": "open", "file": "/tmp/gdsfactory/via$10.gds"}
